## Preprocessing

In [37]:
import keras_tuner as kt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [116]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [117]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN", "NAME"], axis=1, inplace=True)

In [118]:
def apply_cutoff_to_column(df: pd.DataFrame,
                           column: str,
                           cutoff_value: int,
                           new_val_name: str="Other") -> pd.DataFrame:
    """Replaces (Bins) values in the `column` where the
     count is < `cutoff_value` with `new_val_name`.

    :param df: DataFrame with column
    :param column: Column name
    :param cutoff_value: count of values to cutoff
    :param new_val_name: string to replace cutoff values with; default=Other
    :return: DataFrame with replaced values
    """
    _col_unique = df[column].unique()
    _col_value_counts = df[column].value_counts()
    _vals_to_replace = [x for x in _col_unique if _col_value_counts[x] < cutoff_value]
    print(f"Replace with '{new_val_name}': {_vals_to_replace}")

    # Replace in DataFrame
    for app in _vals_to_replace:
        df[column] = df[column].replace(app, new_val_name)

    return df

In [119]:
# APPLICATION_TYPE cutoff = 500
app_df_cutoffs = apply_cutoff_to_column(application_df, "APPLICATION_TYPE", 500)
app_df_cutoffs['APPLICATION_TYPE'].value_counts()

Replace with 'Other': ['T2', 'T9', 'T13', 'T12', 'T29', 'T25', 'T14', 'T17', 'T15']


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [120]:
# CLASSIFICATION cutoff = 700
app_df_cutoffs = apply_cutoff_to_column(app_df_cutoffs, "CLASSIFICATION", 700)
app_df_cutoffs['CLASSIFICATION'].value_counts()

Replace with 'Other': ['C2700', 'C7200', 'C1700', 'C4000', 'C7100', 'C2800', 'C6000', 'C1238', 'C5000', 'C7120', 'C1800', 'C4100', 'C1400', 'C1270', 'C2300', 'C8200', 'C1500', 'C7210', 'C1300', 'C1230', 'C1280', 'C1240', 'C2710', 'C2561', 'C1250', 'C8000', 'C1245', 'C1260', 'C1235', 'C1720', 'C1257', 'C4500', 'C2400', 'C8210', 'C1600', 'C1278', 'C1237', 'C4120', 'C2170', 'C1728', 'C1732', 'C2380', 'C1283', 'C1570', 'C2500', 'C1267', 'C3700', 'C1580', 'C2570', 'C1256', 'C1236', 'C1234', 'C1246', 'C2190', 'C4200', 'C0', 'C3200', 'C5200', 'C1370', 'C2600', 'C1248', 'C6100', 'C1820', 'C1900', 'C2150']


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [121]:
display(application_df["AFFILIATION"].value_counts())
display(application_df["USE_CASE"].value_counts())
display(application_df["ORGANIZATION"].value_counts())
display(application_df["INCOME_AMT"].value_counts())
application_df["SPECIAL_CONSIDERATIONS"].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

Try adding additional cutoffs for `AFFILIATION` and `INCOME_AMT`

In [122]:
# INCOME_AMT cutoff = 500
app_df_cutoffs = apply_cutoff_to_column(app_df_cutoffs, "INCOME_AMT", 500, "5M+")
app_df_cutoffs['INCOME_AMT'].value_counts()

Replace with '5M+': ['10M-50M', '50M+', '5M-10M']


0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
5M+                564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [123]:
# AFFILIATION cutoff = 1000
app_df_cutoffs = apply_cutoff_to_column(app_df_cutoffs, "AFFILIATION", 1000)
app_df_cutoffs['AFFILIATION'].value_counts()

Replace with 'Other': ['Family/Parent', 'National', 'Regional', 'Other']


Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

Drop 'SPECIAL_CONSIDERATIONS' instead of making it a dummy, Special Considerations could be very broad and it is only true for a tiny percentage (<.1%) of organizations.

In [124]:
# Drop SPECIAL_CONSIDERATIONS
application_df.drop("SPECIAL_CONSIDERATIONS", axis=1, inplace=True)

In [125]:
# Convert categorical data to numeric with `pd.get_dummies`
df_app_clean = pd.get_dummies(application_df, columns=["AFFILIATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT",
                                                       "CLASSIFICATION", "APPLICATION_TYPE"])
df_app_clean.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,...,CLASSIFICATION_Other,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,108590,1,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1,6692,1,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,1,142590,1,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [126]:
# Split our preprocessed data into our features and target arrays
X = df_app_clean.drop("IS_SUCCESSFUL", axis=1).values
y = df_app_clean["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instances
# Fit the StandardScaler
scaler = StandardScaler().fit(X_train)

# Scale the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train.shape

(25724, 37)

## Compile, Train and Evaluate the Model

In [127]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_features = X_train.shape[1]
print("Features:", num_features)

nn = tf.keras.models.Sequential()

# number of neurons should be 2-3 times number of inputs=number of features
# there is only 1 output
nn.add(tf.keras.layers.Dense(units=num_features*2, activation="relu", input_dim=num_features))
nn.add(tf.keras.layers.Dense(units=num_features*2, activation="relu"))
nn.add(tf.keras.layers.Dense(units=num_features*2, activation="relu"))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()

Features: 37
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 74)                2812      
                                                                 
 dense_4 (Dense)             (None, 74)                5550      
                                                                 
 dense_5 (Dense)             (None, 74)                5550      
                                                                 
 dense_6 (Dense)             (None, 1)                 75        
                                                                 
Total params: 13,987
Trainable params: 13,987
Non-trainable params: 0
_________________________________________________________________


In [128]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [129]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5677 - accuracy: 0.7241
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5537 - accuracy: 0.7292
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5504 - accuracy: 0.7332
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5487 - accuracy: 0.7341
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5479 - accuracy: 0.7343
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5464 - accuracy: 0.7342
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5455 - accuracy: 0.7358
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5448 - accuracy: 0.7362
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5441 - accuracy: 0.7364
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5442 - accuracy: 0.7374

In [130]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5658 - accuracy: 0.7259 - 330ms/epoch - 1ms/step
Loss: 0.5658435225486755, Accuracy: 0.7259474992752075


---
# Try using Keras Tuner

In [113]:
NUM_FEATURES = X_train.shape[1]
print("Features:", NUM_FEATURES)

def create_model(hp: kt.HyperParameters) -> tf.keras.models.Sequential:
    model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice("activation", ["relu", "tanh"])

    # Allow kerastuner to decide number of neurons in first layer
    model.add(tf.keras.layers.Dense(units=hp.Int("first_units", NUM_FEATURES, NUM_FEATURES*3, step=6),
                                    activation=activation,
                                    input_dim=NUM_FEATURES))
    
    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                     NUM_FEATURES, NUM_FEATURES*3, step=6),
                                        activation=activation))
    # output layer
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    model.compile(loss="binary_crossentropy",
                  optimizer='adam',
                  metrics=["accuracy"])
    return model

tuner = kt.Hyperband(create_model,
                     objective="val_accuracy",
                     max_epochs=25,
                     overwrite=True,
                     hyperband_iterations=2)

Features: 40


In [114]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,
             y_train,
             epochs=25,
             validation_data=(X_test_scaled, y_test))

Trial 60 Complete [00h 00m 37s]
val_accuracy: 0.7267638444900513

Best val_accuracy So Far: 0.7278134226799011
Total elapsed time: 00h 15m 46s
INFO:tensorflow:Oracle triggered exit


In [115]:
# Get top 3 model hyperparameters and print the values
top_3_hp = tuner.get_best_hyperparameters(3)
for hparam in top_3_hp:
    print(hparam.values)

# Evaluate the top 3 models against the test dataset
top_3_model = tuner.get_best_models(3)
for model in top_3_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

{'activation': 'relu', 'first_units': 40, 'num_layers': 1, 'units_0': 70, 'units_1': 46, 'units_2': 52, 'tuner/epochs': 25, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 88, 'num_layers': 2, 'units_0': 76, 'units_1': 46, 'units_2': 64, 'tuner/epochs': 25, 'tuner/initial_epoch': 9, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0051'}
{'activation': 'relu', 'first_units': 58, 'num_layers': 1, 'units_0': 94, 'units_1': 58, 'units_2': 100, 'tuner/epochs': 25, 'tuner/initial_epoch': 9, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0022'}
268/268 - 0s - loss: 0.5530 - accuracy: 0.7278 - 432ms/epoch - 2ms/step
Loss: 0.5529869198799133, Accuracy: 0.7278134226799011
268/268 - 0s - loss: 0.5508 - accuracy: 0.7277 - 467ms/epoch - 2ms/step
Loss: 0.5508202314376831, Accuracy: 0.7276967763900757
268/268 - 0s - loss: 0.5513 - accuracy: 0.7276 - 409ms/epoch - 2ms/step
Loss: 0.5513094067573547, Accuracy: 0.727580189704895


## Save the Best Final Model

In [47]:
# Export our model to HDF5 file
nn.save("models/AlphabetSoupCharity_Optimization.h5")

## Attempts to Improve Accuracy
- Drop `SPECIAL_CONSIDERATIONS` column as it had <1% of value with a 1 instead of a 0
- Change the cutoff values for
    - `APPLICATION_TYPE` to 500
    - `CLASSIFICATION` to 700
    - `INCOME_AMT` to 500, which grouped all 5M+ into one group
- Used `keras_tuner` to attempt hyperparameter optimization